Rough implementations of various stepwise regression techniques

# Package setup

## Import libraries

In [1]:
#@title
# PATH MANAGEMENT
# OS-independent setup
import os
from pathlib import Path
# https://docs.python.org/3/library/pathlib.html
CWD = Path.cwd() / '..'
HOME = Path.home()

USE_COLAB = False

if USE_COLAB:
    CWD = Path(CWD / 'drive' / 'Colab Notebooks')
    print(f"We're running on Colab... alternative CWD: {CWD}")

DATA_IN = CWD / 'data_in'
DATA_OUT = CWD / 'data_out'
VIZ_OUT = CWD / "viz"

# os.chdir(HOME / 'LocalDev' / 'data-analysis-template')

print(f"""PATH SETUP: 
---
cwd is {CWD}

Refer to these paths for IO operations:
HOME: {HOME}
DATA_IN: {DATA_IN}
DATA_OUT: {DATA_OUT}
VIZ_OUT: {VIZ_OUT}
""")

PATH SETUP: 
---
cwd is /Volumes/SanDisk_APFS_EXT/DSMUS_PS3/scripts/..

Refer to these paths for IO operations:
HOME: /Users/riledigital
DATA_IN: /Volumes/SanDisk_APFS_EXT/DSMUS_PS3/scripts/../data_in
DATA_OUT: /Volumes/SanDisk_APFS_EXT/DSMUS_PS3/scripts/../data_out
VIZ_OUT: /Volumes/SanDisk_APFS_EXT/DSMUS_PS3/scripts/../viz



In [2]:
# Importing analysis libraries
import datetime
import pandas as pd
import numpy as np

# Data Viz
# good ol matplotlib
import matplotlib.pyplot as plt
# Easier mplt's
import seaborn as sns
# Vega-Lite for interactives!!!!
import altair as alt
from IPython.core.display import display, HTML
import sklearn as sk
from patsy import dmatrices, dmatrix, demo_data
import statsmodels.api as sm

## Jupyter notebook options and viz options

In [3]:
# Options for python notebook
pd.options.display.max_rows = 1000
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Theming for plots + high DPI figures
%config InlineBackend.figure_format = 'retina'
from io import StringIO

In [4]:
# Helper function to clean column names :-)
# https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd
def clean_column_names(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    return df

## Load Boston Housing Dataset

In [5]:
from sklearn.datasets import load_boston

boston = load_boston()
df_boston = pd.DataFrame(
    boston.data,
    columns=boston.feature_names
)
df_boston['y'] = boston.target
df_boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,y
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


# Helper functions for automated stepwise regressions

## `quick_fit` outputs a dict ready to be turned into a DataFrame for summaries.

In [6]:
from statsmodels.stats.outliers_influence import OLSInfluence

def quick_fit(y, X):
    """ 
    helper func to fit regressions. returns dictionary with relevant stats 
    Ri Le 2020. Please do not share— working on a package.
    """
    X = sm.add_constant(X)
    ols = sm.OLS(y, X)
    results = ols.fit()
    residuals = results.resid
    n = results.nobs     
    p = len(results.params)
    combo = X.columns.values.tolist()
    rss = results.ssr
    rss_textbook = (residuals ** 2).sum()
    msres = results.ssr / (n - p)
    hat_sigma_squared = msres
    c_p = (rss / hat_sigma_squared) - (n + (2 * p))
    reg_row = {
        # recall that k is count of regressors
        'k_count': p - 1,
        'p_count': p,
        'formula': tuple(combo),
        'ss_res': results.ssr,
        'R2': results.rsquared,
        'R2_adj': results.rsquared_adj,
        'MS_res': msres,
        'C_p': c_p,
        'PRESS': OLSInfluence(results).ess_press,
        'AIC': results.aic
    }
    reg_row['model_results'] = results
    reg_row['pt_onevar'] = results.pvalues[0]
    return(reg_row)


### Interactive Stepwise Regression Viewer
Mouseover to view info about the regression.
## `view_stepwise` renders an interactive Altair plot to explore the regression steps.

In [7]:
def view_stepwise(indf):
    """Wrapper to Altair which enables viewing of regression steps
    """
    step_summary_df = indf.drop(['model_results', 'pt_onevar'], axis=1)
    step_summary_df = step_summary_df.reset_index()
    step_summary_df['p_count'] = step_summary_df['p_count'].astype('str')
    cols_viz = step_summary_df.columns.values.tolist()

    stepwise_viewer = alt.Chart(step_summary_df).mark_point(size=300).encode(
        x='index',
        y=alt.Y(alt.repeat("row"), type='quantitative'),
        color='p_count',
        tooltip= cols_viz
    ).properties(
        title='Stepwise Regression Results Viewer',
        height=150,
    ).repeat(
        row=cols_viz,
        spacing=0
    ).add_selection().interactive()
    return(stepwise_viewer)

## `exhaustive_regression` generates every possible regression and generates summary stats using `quick_fit`

In [8]:
from itertools import combinations as combos

def exhaustive_regression(df_in, p_stop=None, helper=True):
    """Automatic exhaustive regression. 
    p_stop: generate models up to p_stop
    """
    y = df_in['y']
    X = df_in.iloc[:,1:]
#     type(y)
#     type(X)
#     print(f"y shape is {y.shape}, X is {X.shape}... \n X cols: {X}. \n y: {y}")
    cols_full = df_in.columns.values.tolist()
    one = combos(cols_full, 2)
    cmap = dict()

    # key is p
    # value is a list of all the combinations for p
    if p_stop is None:
        p_stop = len(cols_full) + 1
    for p in range(1, p_stop):
        all_combos = combos(cols_full, p)
    #     print(p)
        cmap[p] = []
        for combo in all_combos:
            # cast to a list
            combo = combo
            cmap[p].append(combo)
    model_results = {
    }

    mstats = {}
    allmods = {}

    ## STORE RESULTS in a list of dict
    dictlist = []
    
    def calculate_mallows_cp(results, MS_res_full):
        """ Helper function for mallow's c_p """
        n = results.nobs
        p = len(results.params)
        ss_res = results.ssr
        hat_sigma_squared = MS_res_full
        c_p = (ss_res / hat_sigma_squared) - n + 2 * p
        return(c_p)

    # print(mstat)
    # Loop thru all params
    for pval in cmap.keys():
        # Loop thru all formulae
        for combo in cmap[pval]:
            X = df_in[list(combo)]
            reg_row = quick_fit(y, X)
            dictlist.append(reg_row)

    exhaust_table = pd.DataFrame(
        dictlist
    ).reset_index()
#     display(results['formula'].str.contains('y'))
    # Drop ridiculous results that have "y" in them...
    exhaust_table = exhaust_table[~exhaust_table['formula'].apply(lambda x: 'y' in x)]
    ## Calculate mallow's cp again for all of them
    ## Calculate the real Mallow's C_p once we have MS_Res for full model.
    max_row = exhaust_table.loc[exhaust_table['p_count'].idxmax()]
    MS_res_full = max_row['MS_res']
    exhaust_table['C_p'] = exhaust_table['model_results'].apply(lambda x: calculate_mallows_cp(x, MS_res_full))
    if not helper:
        return(exhaust_table.drop(['model_results', 'pt_onevar'], axis=1))
    return(exhaust_table)



## `forward_selection` implements forward selection similar to ISL 4e

In [9]:
def forward_selection(dfb1):
    """ 

    This variation of forward selection looks for improvements of R2_adj rather than t-tests...
    Implementation of Forward Selection as described in ISL 
    http://faculty.marshall.usc.edu/gareth-james/ISL/
    """
    y = dfb1['y']
    X = dfb1.drop('y', axis=1)
    alpha_in = 0.10
    all_cols = set(dfb1.columns.values.tolist().copy())
    all_cols_list = list(all_cols)
    all_cols.remove('y')
    k = len(all_cols)
    # Remove the y
    print(all_cols)
    m0 = []
    candidate_regressors = all_cols.copy()
    acc_regressors = set()
    # First we get all models with m0_k + 1 regressors
    ## Build up a list of good models to print at end
    step_results = []
    def get_best_model_from_subset(candidate_regressors2):
        candmodels = []
        for col in candidate_regressors2:
                candidate_formula = acc_regressors.copy()
                candidate_formula.add(col)
                modresult = quick_fit(y, dfb1[candidate_formula])
                candmodels.append(modresult)
        candsummary = pd.DataFrame(candmodels)
        # get index of top R2adj
    #     display(candsummary)
        maxidx = candsummary['R2_adj'].idxmax() 
        # Strongest formula for the round
        formula_best = list(candsummary.iloc[maxidx]['formula'])
        row_best = candsummary.iloc[maxidx]
        best_r2adj = row_best.R2_adj
        formula_best.remove('const')
        display(f'{formula_best} with R2_adj={best_r2adj :.2f}')
        step_results.append(quick_fit(y, X[formula_best]))
        return([formula_best, best_r2adj])


    def update_state(acc_regressors2, cand_regressors, formula_best):
        # We add X8 to our forward model...
        acc_regressors2.update(formula_best)
        # and we remove some from the candidate_regressors
        cand_regressors.difference_update(formula_best)
        print(f'accumulated: {acc_regressors2}')
        print(f'remaining: {cand_regressors}')
        return([acc_regressors2, cand_regressors, formula_best])
    
    # implementation run that calls all helpers
    best_r2 = 0.0
    while len(candidate_regressors) > 0:
        next_formula, output_r2 = get_best_model_from_subset(candidate_regressors)
        if output_r2 > best_r2:
            best_r2 = output_r2
    #         display([next_formula, output_r2])
            acc_regressors, candidate_regressors, finalbest = update_state(
                acc_regressors, 
                candidate_regressors,
                next_formula
            )
        else:
            print(f'Ended on best formula:\n{next_formula} \nwith R2_Adj={output_r2}')
            display(quick_fit(y, X[next_formula]))
            break

    finalbest
    step_results_summary = pd.DataFrame(step_results)
#     display(step_results_summary)
    return(step_results_summary)

#     t1, t2, t3 = update_state(acc_regressors, candidate_regressors, get_best_model_from_subset(candidate_regressors)[0] )
forward_summary = forward_selection(df_boston)

forward_summary

{'CRIM', 'NOX', 'RAD', 'CHAS', 'DIS', 'ZN', 'AGE', 'TAX', 'RM', 'PTRATIO', 'LSTAT', 'INDUS', 'B'}


"['LSTAT'] with R2_adj=0.54"

accumulated: {'LSTAT'}
remaining: {'CRIM', 'NOX', 'RM', 'CHAS', 'DIS', 'ZN', 'AGE', 'TAX', 'RAD', 'PTRATIO', 'INDUS', 'B'}


"['RM', 'LSTAT'] with R2_adj=0.64"

accumulated: {'RM', 'LSTAT'}
remaining: {'CRIM', 'NOX', 'CHAS', 'DIS', 'ZN', 'AGE', 'TAX', 'RAD', 'PTRATIO', 'INDUS', 'B'}


...

accumulated: {'CRIM', 'NOX', 'RAD', 'CHAS', 'DIS', 'ZN', 'TAX', 'RM', 'PTRATIO', 'LSTAT', 'B'}
remaining: {'AGE', 'INDUS'}


"['CRIM', 'NOX', 'RAD', 'CHAS', 'DIS', 'ZN', 'TAX', 'RM', 'PTRATIO', 'LSTAT', 'INDUS', 'B'] with R2_adj=0.73"

Ended on best formula:
['CRIM', 'NOX', 'RAD', 'CHAS', 'DIS', 'ZN', 'TAX', 'RM', 'PTRATIO', 'LSTAT', 'INDUS', 'B'] 
with R2_Adj=0.7343282238499185


{'k_count': 12,
 'p_count': 13,
 'formula': ('const',
  'CRIM',
  'NOX',
  'RAD',
  'CHAS',
  'DIS',
  'ZN',
  'TAX',
  'RM',
  'PTRATIO',
  'LSTAT',
  'INDUS',
  'B'),
 'ss_res': 11078.846412308365,
 'R2': 0.7406412165505145,
 'R2_adj': 0.7343282238499185,
 'MS_res': 22.472305095960174,
 'C_p': -39.0,
 'PRESS': 11927.413936096598,
 'AIC': 3023.6114182206766,
 'model_results': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x122ef8b90>,
 'pt_onevar': 2.7154637954229124e-12}

,k_count,p_count,formula,ss_res,R2,R2_adj,MS_res,C_p,PRESS,AIC,model_results,pt_onevar
0,1,2,"(const, LSTAT)",19472.381418,0.544146,0.543242,38.635677,-6.0,19678.389502,3286.974957,<statsmodels.regression.linear_model.Regressio...,3.743081e-236
1,2,3,"(const, RM, LSTAT)",15439.309201,0.638562,0.637124,30.694452,-9.0,15814.872540,3171.542314,<statsmodels.regression.linear_model.Regressio...,6.687649e-01
2,3,4,"(const, RM, PTRATIO, LSTAT)",13727.985314,0.678624,0.676704,27.346584,-12.0,14117.504004,3114.097267,<statsmodels.regression.linear_model.Regressio...,2.725808e-06
3,4,5,"(const, RM, PTRATIO, LSTAT, DIS)",13228.907703,0.690308,0.687835,26.405005,-15.0,13696.092387,3097.359045,<statsmodels.regression.linear_model.Regressio...,3.768552e-09
4,5,6,"(const, NOX, DIS, RM, PTRATIO, LSTAT)",12469.344151,0.708089,0.705170,24.938688,-18.0,12973.603871,3069.438633,<statsmodels.regression.linear_model.Regressio...,3.426786e-15
5,6,7,"(const, NOX, PTRATIO, RM, CHAS, DIS, LSTAT)",12141.072736,0.715774,0.712357,24.330807,-21.0,12762.559644,3057.939050,<statsmodels.regression.linear_model.Regressio...,4.291836e-15
6,7,8,"(const, NOX, PTRATIO, RM, CHAS, DIS, LSTAT, B)",11868.235607,0.722161,0.718256,23.831798,-24.0,12541.058642,3048.438383,<statsmodels.regression.linear_model.Regressio...,1.187252e-09
7,8,9,"(const, ZN, NOX, PTRATIO, RM, CHAS, DIS, LSTAT...",11678.299470,0.726608,0.722207,23.497584,-27.0,12389.456369,3042.274993,<statsmodels.regression.linear_model.Regressio...,1.030083e-09
8,9,10,"(const, CRIM, NOX, CHAS, DIS, ZN, RM, PTRATIO,...",11583.587544,0.728825,0.723905,23.354007,-30.0,12370.746205,3040.154562,<statsmodels.regression.linear_model.Regressio...,2.759069e-09
9,10,11,"(const, CRIM, NOX, RAD, CHAS, DIS, ZN, RM, PTR...",11354.983231,0.734177,0.728807,22.939360,-33.0,12156.073414,3032.068702,<statsmodels.regression.linear_model.Regressio...,2.972306e-11


In [10]:
view_stepwise(forward_summary)

alt.RepeatChart(...)

## 1b) Use the backward elimination algorithm to select a subset regression model. Use $\alpha= 0.10$.

In [11]:
def auto_backward_elim(df, alpha_out=0.10 / 2):
    """ implementation of backward elimination
  
    Keyword arguments:
    indf: DataFrame
    alpha_out: variables get rejected if they are not < alpha_out
    """
    ## TODO: backward elimination
    y = df[['y']]
    X = df.drop('y', axis=1)
    all_cols = set(df.columns.values.tolist().copy())
    all_cols_list = list(all_cols)
    all_cols.remove('y')
    ## Start with full model in backward elim
    acc_regressors = set(all_cols.copy())
    alpha_out = 0.10 / 2
    step_results = []
    while len(acc_regressors) >= 1 :
        print(f"""
        Rerunning model: {acc_regressors}
        """)
        X = df[acc_regressors]
        results = quick_fit(y, X)
        sans_constant_t = results['model_results'].tvalues.drop('const')
        sans_constant = results['model_results'].pvalues.drop('const')
        ## Check for highest t val
        ## drop if > alpha /2
        tvalues = sans_constant_t
        pvalues = sans_constant
        largest_p = pvalues.abs().idxmax()
        largest_p_value = pvalues[largest_p]
        # remove_var = results[smallest]
        print(f'Variable with largest p-value is {largest_p}, p-value is {largest_p_value:.2f}')
        results['model_results'].summary()
        print(f'is p {largest_p_value:.2f} greater than alpha_out = {alpha_out}? {largest_p_value > alpha_out}')
        step_results.append(results)
        if largest_p_value > alpha_out:
            print(f'Dropping from model {largest_p} with pvalue= {largest_p_value:.2f}')
            acc_regressors = acc_regressors.difference([largest_p])
        else:
            print(f'No more vars to drop! Ending on {acc_regressors}')
            break
    step_results = pd.DataFrame(step_results)
    return(step_results)
#     return(results['model_results'].summary())

In [12]:
auto_backward_elim(df_boston)


        Rerunning model: {'CRIM', 'NOX', 'RM', 'CHAS', 'DIS', 'ZN', 'AGE', 'TAX', 'RAD', 'PTRATIO', 'LSTAT', 'INDUS', 'B'}
        
Variable with largest p-value is AGE, p-value is 0.96
is p 0.96 greater than alpha_out = 0.05? True
Dropping from model AGE with pvalue= 0.96

        Rerunning model: {'CRIM', 'NOX', 'RM', 'CHAS', 'DIS', 'ZN', 'TAX', 'RAD', 'PTRATIO', 'LSTAT', 'INDUS', 'B'}
        
Variable with largest p-value is INDUS, p-value is 0.74
is p 0.74 greater than alpha_out = 0.05? True
Dropping from model INDUS with pvalue= 0.74

        Rerunning model: {'CRIM', 'NOX', 'RAD', 'CHAS', 'DIS', 'ZN', 'TAX', 'RM', 'PTRATIO', 'LSTAT', 'B'}
        
Variable with largest p-value is CHAS, p-value is 0.00
is p 0.00 greater than alpha_out = 0.05? False
No more vars to drop! Ending on {'CRIM', 'NOX', 'RAD', 'CHAS', 'DIS', 'ZN', 'TAX', 'RM', 'PTRATIO', 'LSTAT', 'B'}


,k_count,p_count,formula,ss_res,R2,R2_adj,MS_res,C_p,PRESS,AIC,model_results,pt_onevar
0,13,14,"(const, CRIM, NOX, RM, CHAS, DIS, ZN, AGE, TAX...",11078.784578,0.740643,0.733790,22.517855,-42.0,12005.227233,3025.608594,<statsmodels.regression.linear_model.Regressio...,3.283438e-12
1,12,13,"(const, CRIM, NOX, RM, CHAS, DIS, ZN, TAX, RAD...",11078.846412,0.740641,0.734328,22.472305,-39.0,11927.413936,3023.611418,<statsmodels.regression.linear_model.Regressio...,2.715464e-12
2,11,12,"(const, CRIM, NOX, RAD, CHAS, DIS, ZN, TAX, RM...",11081.363952,0.740582,0.734806,22.431911,-36.0,11897.702884,3021.726388,<statsmodels.regression.linear_model.Regressio...,2.727265e-12


### Implementing automatic stepwise regression...

- Implemented using recursion with an accumulator. Call `Stepwise()` with parameters

In [13]:
def filter_high_p_values(y_in, X_in, acc_regs, alpha_out = 0.1/2):
    """ Returns a new set of regressors, using quick_fit to check for high p values
    Ri Le 2020. Please do not share— working on a package.
    """
    print(f'acc: {acc_regs}')
    ## Check if we want to remove any now
    # filter out variables whose p values do not meet alpha_out
    results = quick_fit(y_in, X_in[acc_regs])['model_results']
    pvals = results.pvalues.drop('const')
    bool_filter = pvals.apply(lambda x: x > alpha_out)
    new_pvalues = pvals
    new_pvalues = new_pvalues[~bool_filter]
    newformula = new_pvalues.index.tolist()
    display(new_pvalues)
    if acc_regs.intersection(set(newformula)) == acc_regs:
        print(f'None dropped, continuing stepwise')
    else:
        symdif = acc_regs.symmetric_difference(set(newformula))
        drop_var = list(symdif)[0]
        try: 
            list(set(acc_regs).remove(drop_var))
        except:
            pass
#         acc_regs.remove(drop_var)
        print(f'Dropping: {drop_var}')

    #     all_regressors.remove('x_2')
    return(acc_regs)


In [14]:
def test_next_models(y_in, X_in, acc_regs, remaining):
    """
    Generate k+1 models for stepwise regression.
    """
    test_list = []
    for new_var in remaining:
        new_formula = acc_regs.copy()
        new_formula.add(new_var)
        test_list.append(quick_fit(y_in, X_in[new_formula]))
    # Get the top model... 
    test_summary = pd.DataFrame(test_list)
    return(test_summary)

In [15]:
def auto_hybrid_step(df_in, alpha_in=0.10 / 2, alpha_out=0.10 / 2):
    """ implementation of hybrid forward selection.
    Keyword arguments:
    indf: DataFrame
    alpha_out: variables get rejected if they are not < alpha_out
    """
    ## TODO: backward elimination
    y = df_in[['y']]
    X = df_in.drop('y', axis=1)
    all_cols = set(X.columns.values.tolist().copy())
    all_cols_list = list(all_cols)
    
    ## Start with empty
    acc_regressors = set()
    remaining_regressors = all_cols.copy()
    step_results = []
    
    # Find largest simple correlation.
    cormatrix = df_in.corr()[['y']].drop('y').abs()
    var_start_index = df_in.corr()[['y']].drop('y').abs().idxmax()
    var_start = cormatrix.loc[var_start_index].index.values[0]

    acc_regressors.add(var_start)
    remaining_regressors.remove(var_start)
    
    firstmodel = quick_fit(y, X[var_start])
    firstmodel['model_results'].summary()
    step_results.append(firstmodel)
    print(f'accumulated: {acc_regressors}, remaining: {remaining_regressors}')
    # acc_regressors.add(new_var)
    # all_regressors.remove(new_var)
        
    while len(remaining_regressors) > 1 :
        print(f"""
        Searching best models for k+1: {acc_regressors}
        """)
        test_summary = test_next_models(y, X, acc_regressors, remaining_regressors)
        ## Find the best model by R2
        idxmax = test_summary['R2_adj'].idxmax()
        max_row = test_summary.loc[[idxmax]]
        new_form = set(max_row['formula'].values.tolist()[0])
        new_form.remove('const')
        new_var = acc_regressors.symmetric_difference(new_form)
        print(f'new_var is {new_var}')
        new_var = list(new_var)[0]
#         max_row.model_results.values[0].summary()
        
        ## Add the new regressor, remove from incoming
        acc_regressors.add(new_var)
        remaining_regressors.remove(new_var)
        ## Filter out vars that don't meet alpha out
        step_results.append(quick_fit(y, X[acc_regressors]))
        acc_regressors = filter_high_p_values(y, X, acc_regressors)

    step_results = pd.DataFrame(step_results)
    return(step_results)
#     return(results['model_results'].summary())

In [16]:
# auto_hybrid_step(dfb3)
hybrid_summary = auto_hybrid_step(df_boston)
hybrid_summary

accumulated: {'LSTAT'}, remaining: {'CRIM', 'NOX', 'RAD', 'CHAS', 'DIS', 'ZN', 'AGE', 'TAX', 'RM', 'PTRATIO', 'INDUS', 'B'}

        Searching best models for k+1: {'LSTAT'}
        
new_var is {'RM'}
acc: {'RM', 'LSTAT'}


RM       3.472258e-27
LSTAT    6.669365e-41
dtype: float64

None dropped, continuing stepwise

        Searching best models for k+1: {'RM', 'LSTAT'}
        
new_var is {'PTRATIO'}
acc: {'RM', 'PTRATIO', 'LSTAT'}


RM         7.734793e-24
PTRATIO    1.644660e-14
LSTAT      7.944208e-36
dtype: float64

None dropped, continuing stepwise

        Searching best models for k+1: {'RM', 'PTRATIO', 'LSTAT'}
        
new_var is {'DIS'}
acc: {'RM', 'PTRATIO', 'LSTAT', 'DIS'}


...

CRIM       1.010438e-03
NOX        1.209413e-06
RAD        2.996799e-06
CHAS       1.551469e-03
DIS        6.837043e-15
ZN         7.542759e-04
TAX        5.214237e-04
RM         2.889779e-19
PTRATIO    9.235063e-13
LSTAT      2.140586e-25
B          5.565743e-04
dtype: float64

None dropped, continuing stepwise

        Searching best models for k+1: {'CRIM', 'NOX', 'RAD', 'CHAS', 'DIS', 'ZN', 'TAX', 'RM', 'PTRATIO', 'LSTAT', 'B'}
        
new_var is {'INDUS'}
acc: {'CRIM', 'NOX', 'RAD', 'CHAS', 'DIS', 'ZN', 'TAX', 'RM', 'PTRATIO', 'LSTAT', 'INDUS', 'B'}


CRIM       1.074747e-03
NOX        1.967110e-06
RAD        4.750539e-06
CHAS       1.862634e-03
DIS        5.027955e-14
ZN         7.193806e-04
TAX        1.099120e-03
RM         3.365945e-19
PTRATIO    1.099178e-12
LSTAT      2.569688e-25
B          5.444689e-04
dtype: float64

Dropping: INDUS


,k_count,p_count,formula,ss_res,R2,R2_adj,MS_res,C_p,PRESS,AIC,model_results,pt_onevar
0,1,2,"(const, LSTAT)",19472.381418,0.544146,0.543242,38.635677,-6.0,19678.389502,3286.974957,<statsmodels.regression.linear_model.Regressio...,3.743081e-236
1,2,3,"(const, RM, LSTAT)",15439.309201,0.638562,0.637124,30.694452,-9.0,15814.872540,3171.542314,<statsmodels.regression.linear_model.Regressio...,6.687649e-01
2,3,4,"(const, RM, PTRATIO, LSTAT)",13727.985314,0.678624,0.676704,27.346584,-12.0,14117.504004,3114.097267,<statsmodels.regression.linear_model.Regressio...,2.725808e-06
3,4,5,"(const, RM, PTRATIO, LSTAT, DIS)",13228.907703,0.690308,0.687835,26.405005,-15.0,13696.092387,3097.359045,<statsmodels.regression.linear_model.Regressio...,3.768552e-09
4,5,6,"(const, NOX, DIS, RM, PTRATIO, LSTAT)",12469.344151,0.708089,0.705170,24.938688,-18.0,12973.603871,3069.438633,<statsmodels.regression.linear_model.Regressio...,3.426786e-15
5,6,7,"(const, NOX, CHAS, DIS, RM, PTRATIO, LSTAT)",12141.072736,0.715774,0.712357,24.330807,-21.0,12762.559644,3057.939050,<statsmodels.regression.linear_model.Regressio...,4.291836e-15
6,7,8,"(const, NOX, CHAS, DIS, RM, PTRATIO, LSTAT, B)",11868.235607,0.722161,0.718256,23.831798,-24.0,12541.058642,3048.438383,<statsmodels.regression.linear_model.Regressio...,1.187252e-09
7,8,9,"(const, NOX, CHAS, DIS, ZN, RM, PTRATIO, LSTAT...",11678.299470,0.726608,0.722207,23.497584,-27.0,12389.456369,3042.274993,<statsmodels.regression.linear_model.Regressio...,1.030083e-09
8,9,10,"(const, CRIM, NOX, CHAS, DIS, ZN, RM, PTRATIO,...",11583.587544,0.728825,0.723905,23.354007,-30.0,12370.746205,3040.154562,<statsmodels.regression.linear_model.Regressio...,2.759069e-09
9,10,11,"(const, CRIM, NOX, RAD, CHAS, DIS, ZN, RM, PTR...",11354.983231,0.734177,0.728807,22.939360,-33.0,12156.073414,3032.068702,<statsmodels.regression.linear_model.Regressio...,2.972306e-11


In [17]:
view_stepwise(hybrid_summary)

alt.RepeatChart(...)